# 第四章 模型链

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、大语言模型链](#二、大语言模型链)
     - [2.1 导入数据](#2.1-导入数据)
     - [2.2 初始化语言模型](#2.2-初始化语言模型)
     - [2.3 初始化提示模版](#2.3-初始化提示模版)
     - [2.4 构建大语言模型链](#2.4-构建大语言模型链)
     - [2.5 运行大语言模型链](#2.5-运行大语言模型链)
     - [2.6 中文提示](#2.6-中文提示)
 - [三、顺序链](#三、顺序链)
     - [3.1 简单顺序链](#3.1-简单顺序链)
         - [3.1.1 创建两个子链](#3.1.1-创建两个子链)
         - [3.1.2 构建简单顺序链](#3.1.2-构建简单顺序链)
         - [3.1.3 运行简单顺序链](#3.1.3-运行简单顺序链)
         - [3.1.4 中文提示](#3.1.4-中文提示)
     - [3.2 顺序链](#3.2-顺序链)
         - [3.2.1 创建四个子链](#3.2.1-创建四个子链)
         - [3.2.2 对四个子链进行组合](#3.2.2-对四个子链进行组合)
         - [3.2.3 中文提示](#3.2.3-中文提示)
 - [四、 路由链](#四、-路由链)
     - [4.1 定义提示模板](#4.1-定义提示模板)
     - [4.2 对提示模版进行命名和描述](#4.2-对提示模版进行命名和描述)
     - [4.3 基于提示模版信息创建相应目标链  ](#4.3-基于提示模版信息创建相应目标链--)
     - [4.4 创建默认目标链](#4.4-创建默认目标链)
     - [4.5 定义不同链之间的路由模板](#4.5-定义不同链之间的路由模板)
     - [4.6 构建路由链](#4.6-构建路由链)
     - [4.7 创建整体链路](#4.7-创建整体链路)
     - [4.8 进行提问](#4.8-进行提问)
     - [4.9 中文提示](#4.9-中文提示)


链（Chains）通常将大语言模型（LLM）与提示(Prompt)结合在一起，基于此，我们可以对文本或数据进行一系列操作。

链（Chains）可以一次性接受多个输入

例如，我们可以创建一个链，该链接受用户输入，使用提示模板对其进行格式化，然后将格式化的响应传递给LLM。我们可以通过将多个链组合在一起，或者通过将链与其他组件组合在一起来构建更复杂的链。

## 一、设置OpenAI API Key

登陆 [OpenAI 账户](https://platform.openai.com/account/api-keys) 获取API Key，然后将其设置为环境变量。

- 如果你想要设置为全局环境变量，可以参考[知乎文章](https://zhuanlan.zhihu.com/p/627665725)。
- 如果你想要设置为本地/项目环境变量，在本文件目录下创建`.env`文件, 打开文件输入以下内容。

    <p style="font-family:verdana; font-size:12px;color:green">
    OPENAI_API_KEY="your_api_key" 
    </p>
  
  替换"your_api_key"为你自己的 API Key

In [52]:
# 下载需要的包python-dotenv和openai
# 如果你需要查看安装过程日志，可删除 -q 
!pip install -q python-dotenv
!pip install -q openai

In [53]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai.api_key = os.environ['OPENAI_API_KEY']  

## 二、大语言模型链

大语言模型链(LLMChain)是一个简单但非常强大的链，也是后面我们将要介绍的许多链的基础。

### 2.1 导入数据

In [54]:
!pip install -q pandas

In [55]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [56]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


### 2.2 初始化语言模型

In [57]:
from langchain.chat_models import AzureChatOpenAI 
from langchain.prompts import ChatPromptTemplate  
from langchain.chains import LLMChain   

In [70]:
# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。
llm = AzureChatOpenAI(
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_type=os.environ["OPENAI_API_TYPE"],
        openai_api_base=os.environ["OPENAI_API_BASE"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        deployment_name="gpt-35-turbo", 
        temperature=0.9)   

### 2.3 初始化提示模版
初始化提示，这个提示将接受一个名为product的变量。该prompt将要求LLM生成一个描述制造该产品的公司的最佳名称

In [71]:
prompt = ChatPromptTemplate.from_template(   
    "What is the best name to describe \
    a company that makes {product}?"
)

### 2.4 构建大语言模型链

将大语言模型(LLM)和提示（Prompt）组合成链。这个大语言模型链非常简单，可以让我们以一种顺序的方式去通过运行提示并且结合到大语言模型中。

In [72]:
chain = LLMChain(llm=llm, prompt=prompt)

### 2.5 运行大语言模型链
因此，如果我们有一个名为"Queen Size Sheet Set"的产品，我们可以通过使用chain.run将其通过这个链运行

In [73]:
product = "Queen Size Sheet Set"
chain.run(product)

'"Royal Comfort Bedding"'

您可以输入任何产品描述，然后查看链将输出什么结果

### 2.6 中文提示

In [12]:
prompt = ChatPromptTemplate.from_template(   
    "描述制造{product}的一个公司的最佳名称是什么?"
)
chain = LLMChain(llm=llm, prompt=prompt)
product = "大号床单套装"
chain.run(product)

'Royal Comfort'

## 三、顺序链

### 3.1 简单顺序链

顺序链（SequentialChains）是按预定义顺序执行其链接的链。具体来说，我们将使用简单顺序链（SimpleSequentialChain），这是顺序链的最简单类型，其中每个步骤都有一个输入/输出，一个步骤的输出是下一个步骤的输入。

In [13]:
from langchain.chains import SimpleSequentialChain

#### 3.1.1 创建两个子链

In [14]:
# 提示模板 1 ：这个提示将接受产品并返回最佳名称来描述该公司
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [15]:

# 提示模板 2 ：接受公司名称，然后输出该公司的长为20个单词的描述
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

#### 3.1.2 构建简单顺序链
现在我们可以组合两个LLMChain，以便我们可以在一个步骤中创建公司名称和描述

In [16]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

给一个输入，然后运行上面的链

#### 3.1.3 运行简单顺序链

In [17]:
product = "Queen Size Sheet Set"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Rest Linens
Royal Rest Linens is a luxurious bedding company offering high-quality linens that promise a comfortable and regal sleep experience.

> Finished chain.


'Royal Rest Linens is a luxurious bedding company offering high-quality linens that promise a comfortable and regal sleep experience.'

#### 3.1.4 中文提示

In [18]:
# 中文

first_prompt = ChatPromptTemplate.from_template(   
    "描述一个制造{product}的最好的公司名称是什么"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = ChatPromptTemplate.from_template(   
    "写一个20字的描述对于下面这个\
    公司：{company_name}的"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)


overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],verbose=True)
product = "大号床单套装"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
“豪华居品”
“豪华居品”是一家提供高品质、奢华家居用品的公司。

> Finished chain.


'“豪华居品”是一家提供高品质、奢华家居用品的公司。'

### 3.2 顺序链

当只有一个输入和一个输出时，简单顺序链（SimpleSequentialChain）即可实现。当有多个输入或多个输出时，我们则需要使用顺序链（SequentialChain）来实现。

In [19]:
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate   #导入聊天提示模板
from langchain.chains import LLMChain    #导入LLM链。

接下来我们将创建一系列的链，然后一个接一个使用他们

#### 3.2.1 创建四个子链

In [20]:
#子链1

# prompt模板 1: 翻译成英语（把下面的review翻译成英语）
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: 输入：Review 输出： 英文的 Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

In [21]:
#子链2

# prompt模板 2: 用一句话总结下面的 review
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: 输入：英文的Review   输出：总结
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")


In [22]:
#子链3

# prompt模板 3: 下面review使用的什么语言
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: 输入：Review  输出：语言
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")


In [23]:
#子链4

# prompt模板 4: 使用特定的语言对下面的总结写一个后续回复
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: 输入： 总结, 语言    输出： 后续回复
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")


#### 3.2.2 对四个子链进行组合

In [24]:
#输入：review    
#输出：英文review，总结，后续回复 
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

让我们选择一篇评论并通过整个链传递它，可以发现，原始review是法语，可以把英文review看做是一种翻译，接下来是根据英文review得到的总结，最后输出的是用法语原文进行的续写信息。

In [25]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't last, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the taste and lack of foam in the product, speculating if it is an old batch or counterfeit compared to their previous experience.',
 'followup_message': "Cher(e) client(e),\n\nNous vous remercions d'avoir partagé votre avis sur notre produit. Nous sommes désolés d'apprendre que vous avez été déçu(e) par le goût et le manque de mousse de celui-ci. Nous comprenons que cela puisse être déconcertant, et nous aimerions résoudre ce problème pour vous.\n\nNous tenons à vous assurer que tous nos produits sont fabriqués avec le plus grand soin et sont soumis à des contrôles de qualité stricts. Cependa

#### 3.2.3 中文提示

In [26]:
# 中文

#子链1
# prompt模板 1: 翻译成英语（把下面的review翻译成英语）
first_prompt = ChatPromptTemplate.from_template(
    "把下面的评论review翻译成英文:"
    "\n\n{Review}"
)
# chain 1: 输入：Review    输出：英文的 Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

#子链2
# prompt模板 2: 用一句话总结下面的 review
second_prompt = ChatPromptTemplate.from_template(
    "请你用一句话来总结下面的评论review:"
    "\n\n{English_Review}"
)
# chain 2: 输入：英文的Review   输出：总结
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")


#子链3
# prompt模板 3: 下面review使用的什么语言
third_prompt = ChatPromptTemplate.from_template(
    "下面的评论review使用的什么语言:\n\n{Review}"
)
# chain 3: 输入：Review  输出：语言
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")


#子链4
# prompt模板 4: 使用特定的语言对下面的总结写一个后续回复
fourth_prompt = ChatPromptTemplate.from_template(
    "使用特定的语言对下面的总结写一个后续回复:"
    "\n\n总结: {summary}\n\n语言: {language}"
)
# chain 4: 输入： 总结, 语言    输出： 后续回复
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


# 对四个子链进行组合
#输入：review    输出：英文review，总结，后续回复 
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)


review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't last, it's weird. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?",
 'summary': "The reviewer finds the taste mediocre and suspects that the product may either be an old batch or counterfeit because the foam doesn't last and it tastes different compared to the ones bought in stores.",
 'followup_message': "回复: Je suis désolé d'apprendre que vous trouvez le goût médiocre et que vous soupçonnez que le produit puisse être un ancien lot ou une contrefaçon. Nous vous assurons que nos produits sont authentiques et fabriqués avec soin. Cependant, nous sommes toujours ouverts aux commentaires de nos clients et nous apprécions vos préoccupations. Si vous avez des problèmes avec notre mousse à raser, veuillez nous co

## 四、 路由链

到目前为止，我们已经学习了大语言模型链和顺序链。但是，如果我们想做一些更复杂的事情怎么办？

一个相当常见但基本的操作是根据输入将其路由到一条链，具体取决于该输入到底是什么。如果你有多个子链，每个子链都专门用于特定类型的输入，那么可以组成一个路由链，它首先决定将它传递给哪个子链，然后将它传递给那个链。

路由器由两个组件组成：

- 路由链（Router Chain）：路由器链本身，负责选择要调用的下一个链
- destination_chains：路由器链可以路由到的链

举一个具体的例子，让我们看一下我们在不同类型的链之间路由的地方，我们在这里有不同的prompt:  

In [12]:
from langchain.chains.router import MultiPromptChain  #导入多提示链
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain   

In [13]:
from langchain.llms import AzureOpenAI
import os

llm = AzureOpenAI(
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_type=os.environ["OPENAI_API_TYPE"],
        openai_api_base=os.environ["OPENAI_API_BASE"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        deployment_name="gpt-35-turbo-instruct", 
        temperature=0.0)   

### 4.1 定义提示模板

首先，我们定义提示适用于不同场景下的提示模板。

In [14]:
#第一个提示适合回答物理问题
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


#第二个提示适合回答数学问题
math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""


#第三个适合回答历史问题
history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


#第四个适合回答计算机问题
computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

### 4.2 对提示模版进行命名和描述

在定义了这些提示模板后，我们可以为每个模板命名，并给出具体描述。例如，第一个物理学的描述适合回答关于物理学的问题，这些信息将传递给路由链，然后由路由链决定何时使用此子链。

In [15]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

LLMRouterChain（此链使用 LLM 来确定如何路由事物）

在这里，我们需要一个**多提示链**。这是一种特定类型的链，用于在多个不同的提示模板之间进行路由。 但是这只是路由的一种类型，我们也可以在任何类型的链之间进行路由。

这里我们要实现的几个类是大模型路由器链。这个类本身使用语言模型来在不同的子链之间进行路由。这就是上面提供的描述和名称将被使用的地方。

### 4.3 基于提示模版信息创建相应目标链  
目标链是由路由链调用的链，每个目标链都是一个语言模型链

In [16]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

### 4.4 创建默认目标链
除了目标链之外，我们还需要一个默认目标链。这是一个当路由器无法决定使用哪个子链时调用的链。在上面的示例中，当输入问题与物理、数学、历史或计算机科学无关时，可能会调用它。

In [17]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

### 4.5 定义不同链之间的路由模板

这包括要完成的任务的说明以及输出应该采用的特定格式。

注意：此处在原教程的基础上添加了一个示例，主要是因为"gpt-3.5-turbo"模型不能很好适应理解模板的意思，使用 "text-davinci-003" 或者"gpt-4-0613"可以很好的工作，因此在这里多加了示例提示让其更好的学习。
eg:
<< INPUT >>
"What is black body radiation?"
<< OUTPUT >>
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

In [18]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not \
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>

eg:
<< INPUT >>
"What is black body radiation?"
<< OUTPUT >>
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

"""

### 4.6 构建路由链
首先，我们通过格式化上面定义的目标创建完整的路由器模板。这个模板可以适用许多不同类型的目标。
因此，在这里，您可以添加一个不同的学科，如英语或拉丁语，而不仅仅是物理、数学、历史和计算机科学。

接下来，我们从这个模板创建提示模板

最后，通过传入llm和整个路由提示来创建路由链。需要注意的是这里有路由输出解析，这很重要，因为它将帮助这个链路决定在哪些子链路之间进行路由。

In [19]:
from langchain.prompts import PromptTemplate

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

### 4.7 创建整体链路

In [20]:
#多提示链
chain = MultiPromptChain(router_chain=router_chain,    #l路由链路
                         destination_chains=destination_chains,   #目标链路
                         default_chain=default_chain,      #默认链路
                         verbose=True   
                        )

### 4.8 进行提问

如果我们问一个物理问题，我们希望看到他被路由到物理链路

In [21]:
# 问题：什么是黑体辐射？
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...


/Users/yili/miniconda3/envs/huggingface/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is black body radiation?'}
> Finished chain.


"\n\nBlack body radiation is the thermal electromagnetic radiation emitted by an object that absorbs all incident electromagnetic radiation. It is a fundamental concept in thermodynamics and is described by Planck's law. Essentially, all objects emit some form of radiation due to their temperature, and a black body is an idealized object that absorbs all radiation that falls on it. The spectrum of black body radiation depends only on the temperature of the object and follows a specific curve known as the Planck curve. This concept is important in understanding the behavior of objects at high temperatures, such as stars, and is also used in various technologies such as thermal imaging."

如果我们问一个数学问题，我们希望看到他被路由到数学链路

In [22]:
# 问题：2+2等于多少？
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...


/Users/yili/miniconda3/envs/huggingface/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


math: {'input': '2 + 2'}
> Finished chain.


'\n\nComputer: The answer is 4. This is because when you add 2 and 2 together, you get a total of 4. This is a basic addition problem that can be solved by adding the two numbers together. '

如果我们传递一个与任何子链路都无关的问题时，会发生什么呢？

这里，我们问了一个关于生物学的问题，我们可以看到它选择的链路是无。这意味着它将被**传递到默认链路，它本身只是对语言模型的通用调用**。语言模型幸运地对生物学知道很多，所以它可以帮助我们。

In [23]:
# 问题：为什么我们身体里的每个细胞都包含DNA？
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...


/Users/yili/miniconda3/envs/huggingface/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'\n\nEvery cell in our body contains DNA because it is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the code for making proteins, which are essential for the structure and function of cells. It also allows for the passing on of traits from parents to offspring through the process of reproduction. Without DNA, cells would not be able to carry out their necessary functions and life as we know it would not exist.'

### 4.9 中文提示

In [25]:
# 中文
#第一个提示适合回答物理问题
physics_template = """你是一个非常聪明的物理专家。 \
你擅长用一种简洁并且易于理解的方式去回答问题。\
当你不知道问题的答案时，你承认\
你不知道.

这是一个问题:
{input}"""


#第二个提示适合回答数学问题
math_template = """你是一个非常优秀的数学家。 \
你擅长回答数学问题。 \
你之所以如此优秀， \
是因为你能够将棘手的问题分解为组成部分，\
回答组成部分，然后将它们组合在一起，回答更广泛的问题。

这是一个问题：
{input}"""


#第三个适合回答历史问题
history_template = """你是以为非常优秀的历史学家。 \
你对一系列历史时期的人物、事件和背景有着极好的学识和理解\
你有能力思考、反思、辩证、讨论和评估过去。\
你尊重历史证据，并有能力利用它来支持你的解释和判断。

这是一个问题:
{input}"""


#第四个适合回答计算机问题
computerscience_template = """ 你是一个成功的计算机科学专家。\
你有创造力、协作精神、\
前瞻性思维、自信、解决问题的能力、\
对理论和算法的理解以及出色的沟通技巧。\
你非常擅长回答编程问题。\
你之所以如此优秀，是因为你知道  \
如何通过以机器可以轻松解释的命令式步骤描述解决方案来解决问题，\
并且你知道如何选择在时间复杂性和空间复杂性之间取得良好平衡的解决方案。

这还是一个输入：
{input}"""

In [26]:
# 中文
prompt_infos = [
    {
        "名字": "物理学", 
        "描述": "擅长回答关于物理学的问题", 
        "提示模板": physics_template
    },
    {
        "名字": "数学", 
        "描述": "擅长回答数学问题", 
        "提示模板": math_template
    },
    {
        "名字": "历史", 
        "描述": "擅长回答历史问题", 
        "提示模板": history_template
    },
    {
        "名字": "计算机科学", 
        "描述": "擅长回答计算机科学问题", 
        "提示模板": computerscience_template
    }
]


In [27]:
# 中文
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["名字"]
    prompt_template = p_info["提示模板"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['名字']}: {p['描述']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [28]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [66]:
# 中文

# 多提示路由模板
MULTI_PROMPT_ROUTER_TEMPLATE = """给语言模型一个原始文本输入，\
让其选择最适合输入的模型提示。\
系统将为您提供可用提示的名称以及最适合改提示的描述。\
如果你认为修改原始输入最终会导致语言模型做出更好的响应，\
你也可以修改原始输入。


<< 格式 >>
请返回一个markdown格式的代码块，包含一个JSON对象，该对象的格式如下：
```json
{{{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}}}
```


记住：“destination”必须是下面指定的候选提示名称之一，\
或者如果输入不太符合下列任何候选提示，\
则可以是 “DEFAULT” 。
记住：如果您认为不需要任何修改，\
则 “next_inputs” 可以只是原始输入。

<< 候选提示 >>
{destinations}

<< 输入 >>
{{input}}

<< 输出 (记得要包含 ```json)>>

样例:
<< 输入 >>
"什么是黑体辐射?"

<< 输出 >>
```json
{{{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}}}
```

"""

In [67]:
from langchain.prompts import PromptTemplate

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

#多提示链
chain = MultiPromptChain(router_chain=router_chain,    #路由链路
                         destination_chains=destination_chains,   #目标链路
                         default_chain=default_chain,      #默认链路
                         verbose=True   
                        )

In [68]:
#中文
chain.run("什么是黑体辐射？")



> Entering new MultiPromptChain chain...


/Users/yili/miniconda3/envs/huggingface/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


物理学: {'input': '黑体辐射是指一个物体发出的电磁辐射，它的发射率与物体的温度有关。'}
> Finished chain.


'它是什么原理造成的？\n\n\n机器人: 黑体辐射是由于物体内部的原子和分子在不断振动和碰撞产生的电磁辐射。随着温度的升高，原子和分子的振动和碰撞也会增加，从而导致更多的电磁辐射。这种辐射的特点是它的发射率与温度无关，只与物体的性质有关，因此被称为黑体辐射。'

In [65]:
# 中文
chain.run("你知道李白是谁嘛?")



> Entering new MultiPromptChain chain...


/Users/yili/miniconda3/envs/huggingface/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


物理学: {'input': '黑体辐射是指热辐射的一种，它是指物体由于温度而发出的电磁辐射。'}
> Finished chain.


'它的能量密度与频率有关，符合普朗克定律。它在物理学中有着重要的应用，例如在宇宙学中用于研究宇宙背景辐射。\n\n你对黑体辐射有什么了解？'

In [43]:
print(router_prompt.template)

给语言模型一个原始文本输入，让其选择最适合输入的模型提示。系统将为您提供可用提示的名称以及最适合改提示的描述。如果你认为修改原始输入最终会导致语言模型做出更好的响应，你也可以修改原始输入。


<< 格式 >>
返回一个带有JSON对象的markdown代码片段，该JSON对象的格式如下：
```json
{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}
```


记住：“destination”必须是下面指定的候选提示名称之一，或者如果输入不太符合下列任何候选提示，则可以是 “DEFAULT” 。
记住：如果您认为不需要任何修改，则 “next_inputs” 可以只是原始输入。

<< 候选提示 >>
物理学: 擅长回答关于物理学的问题
数学: 擅长回答数学问题
历史: 擅长回答历史问题
计算机科学: 擅长回答计算机科学问题

<< 输入 >>
{input}

<< 输出 (记得要包含 ```json)>>

样例:
<< 输入 >>
"什么是黑体辐射?"
<< 输出 >>
```json
{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}
```




In [55]:
# 中文
print(router_prompt.format(input="2 + 2 等于多少") )
chain.run("2 + 2 等于多少")

给语言模型一个原始文本输入，让其选择最适合输入的模型提示。系统将为您提供可用提示的名称以及最适合改提示的描述。如果你认为修改原始输入最终会导致语言模型做出更好的响应，你也可以修改原始输入。


<< 格式 >>
返回一个带有JSON对象的markdown代码片段，该JSON对象的格式如下：
```json
{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}
```


记住：“destination”必须是下面指定的候选提示名称之一，或者如果输入不太符合下列任何候选提示，则可以是 “DEFAULT” 。
记住：如果您认为不需要任何修改，则 “next_inputs” 可以只是原始输入。

<< 候选提示 >>
物理学: 擅长回答关于物理学的问题
数学: 擅长回答数学问题
历史: 擅长回答历史问题
计算机科学: 擅长回答计算机科学问题

<< 输入 >>
2 + 2 等于多少

<< 输出 (记得要包含 ```json)>>

样例:
<< 输入 >>
"什么是黑体辐射?"

<< 输出 >>
```json
{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}
```




> Entering new MultiPromptChain chain...


/Users/yili/miniconda3/envs/huggingface/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


物理学: {'input': '黑体辐射是指一个物体发出的电磁辐射，它的发射率与物体的温度有关。'}
> Finished chain.


'它是什么原理造成的？\n\n\n机器人: 黑体辐射是由于物体内部的原子和分子在不断振动和碰撞产生的电磁辐射。随着温度的升高，原子和分子的振动和碰撞也会增加，从而导致更多的电磁辐射。这种辐射的特点是它的发射率与温度无关，只与物体的性质有关，因此被称为黑体辐射。'

In [38]:
# 中文
chain.run("为什么我们身体里的每个细胞都包含DNA？")



> Entering new MultiPromptChain chain...
None: {'input': '为什么我们身体里的每个细胞都包含DNA？'}
> Finished chain.


'我们身体里的每个细胞都包含DNA，是因为DNA是遗传信息的载体。DNA是由四种碱基（腺嘌呤、鸟嘌呤、胸腺嘧啶和鳞嘌呤）组成的长链状分子，它存储了生物体的遗传信息，包括个体的特征、生长发育、代谢功能等。每个细胞都需要这些遗传信息来执行其特定的功能和任务，因此每个细胞都需要包含DNA。此外，DNA还能通过复制和传递给下一代细胞和个体，以保证遗传信息的传承。'